In [7]:
import itertools
import jsonlines
import pandas as pd
from pprint import pprint

from datasets import load_dataset

In [8]:
pretrained_dataset = load_dataset('monology/pile-uncopyrighted', split='train', streaming=True)

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [9]:
n = 5
print("Pretrained dataset: ")
top_n = itertools.islice(pretrained_dataset, n)
for i in top_n:
    print(i)

Pretrained dataset: 
{'text': 'It is done, and submitted. You can play “Survival of the Tastiest” on Android, and on the web. Playing on the web works, but you have to simulate multi-touch for table moving and that can be a bit confusing.\n\nThere’s a lot I’d like to talk about. I’ll go through every topic, insted of making the typical what went right/wrong list.\n\nConcept\n\nWorking over the theme was probably one of the hardest tasks I had to face.\n\nOriginally, I had an idea of what kind of game I wanted to develop, gameplay wise – something with lots of enemies/actors, simple graphics, maybe set in space, controlled from a top-down view. I was confident I could fit any theme around it.\n\nIn the end, the problem with a theme like “Evolution” in a game is that evolution is unassisted. It happens through several seemingly random mutations over time, with the most apt permutation surviving. This genetic car simulator is, in my opinion, a great example of actual evolution of a specie

In [12]:
filelink = 'hf://datasets/kotzeje/lamini_docs.jsonl/data/train-00000-of-00001-6359aa989b671345.parquet'
instruction_dataset_df = pd.read_parquet(filelink)
instruction_dataset_df

,question,answer
0,How can I evaluate the performance and quality...,There are several metrics that can be used to ...
1,Can I find information about the code's approa...,"Yes, the code includes methods for submitting ..."
2,How does Lamini AI handle requests for generat...,Lamini AI offers features for generating text ...
3,Does the `submit_job()` function expose any ad...,It is unclear which `submit_job()` function is...
4,Does the `add_data()` function support differe...,"No, the `add_data()` function does not support..."
...,...,...
1395,Does Lamini have the ability to understand and...,"Yes, Lamini has the ability to understand and ..."
1396,Can I fine-tune the pre-trained models provide...,"Yes, you can fine-tune the pre-trained models ..."
1397,Can Lamini generate text that is suitable for ...,"Yes, Lamini can generate text that is suitable..."
1398,Does the documentation have a secret code that...,I wish! This documentation only talks about La...


In [15]:
examples = instruction_dataset_df.to_dict()
text = examples["question"][0] + examples['answer'][0]
text

"How can I evaluate the performance and quality of the generated text from Lamini models?There are several metrics that can be used to evaluate the performance and quality of generated text from Lamini models, including perplexity, BLEU score, and human evaluation. Perplexity measures how well the model predicts the next word in a sequence, while BLEU score measures the similarity between the generated text and a reference text. Human evaluation involves having human judges rate the quality of the generated text based on factors such as coherence, fluency, and relevance. It is recommended to use a combination of these metrics for a comprehensive evaluation of the model's performance."

In [16]:
if 'question' in examples and 'answer' in examples:
    text = examples["question"][0] + examples['answer'][0]
elif 'instruction' in examples and 'response' in examples:
    text = examples["instruction"][0] + examples['response'][0]
elif 'input' in examples and 'output' in examples:
    text = examples["input"][0] + examples['output'][0]
else:
    text = examples["text"][0]

In [21]:
prompt_template_QA = '''### Question:
{question}

### Answer:
{answer}'''

question = examples["question"][0]
answer = examples['answer'][0]
text_with_prompt_template = prompt_template_QA.format(question=question, answer=answer)
text_with_prompt_template

"### Question:\nHow can I evaluate the performance and quality of the generated text from Lamini models?\n\n### Answer:\nThere are several metrics that can be used to evaluate the performance and quality of generated text from Lamini models, including perplexity, BLEU score, and human evaluation. Perplexity measures how well the model predicts the next word in a sequence, while BLEU score measures the similarity between the generated text and a reference text. Human evaluation involves having human judges rate the quality of the generated text based on factors such as coherence, fluency, and relevance. It is recommended to use a combination of these metrics for a comprehensive evaluation of the model's performance."

In [22]:
prompt_template_Q = '''### Question:
{question}

### Answer:'''

In [29]:
num_examples = len(examples['question'])
finetuning_dataset_txt_only = []
finetuning_dataset_QA = []
for i in range(num_examples):
    question = examples['question'][i]
    answer = examples['answer'][i]

    text_with_prompt_template_QA = prompt_template_QA.format(question=question, answer=answer)
    finetuning_dataset_txt_only.append({'text': text_with_prompt_template_QA})
    text_with_prompt_template_Q = prompt_template_Q.format(question=question)
    finetuning_dataset_QA.append({'question': text_with_prompt_template_Q, 'answer': answer})

In [30]:
pprint(finetuning_dataset_txt_only[0])

{'text': '### Question:\n'
         'How can I evaluate the performance and quality of the generated text '
         'from Lamini models?\n'
         '\n'
         '### Answer:\n'
         'There are several metrics that can be used to evaluate the '
         'performance and quality of generated text from Lamini models, '
         'including perplexity, BLEU score, and human evaluation. Perplexity '
         'measures how well the model predicts the next word in a sequence, '
         'while BLEU score measures the similarity between the generated text '
         'and a reference text. Human evaluation involves having human judges '
         'rate the quality of the generated text based on factors such as '
         'coherence, fluency, and relevance. It is recommended to use a '
         'combination of these metrics for a comprehensive evaluation of the '
         "model's performance."}


In [31]:
pprint(finetuning_dataset_QA[0])

{'answer': 'There are several metrics that can be used to evaluate the '
           'performance and quality of generated text from Lamini models, '
           'including perplexity, BLEU score, and human evaluation. Perplexity '
           'measures how well the model predicts the next word in a sequence, '
           'while BLEU score measures the similarity between the generated '
           'text and a reference text. Human evaluation involves having human '
           'judges rate the quality of the generated text based on factors '
           'such as coherence, fluency, and relevance. It is recommended to '
           'use a combination of these metrics for a comprehensive evaluation '
           "of the model's performance.",
 'question': '### Question:\n'
             'How can I evaluate the performance and quality of the generated '
             'text from Lamini models?\n'
             '\n'
             '### Answer:'}


In [32]:
with jsonlines.open(f'lamini_docs_processed.jonsl', mode='w') as writer:
    writer.write_all(finetuning_dataset_QA)